In [1]:
from process_bigraph import Composite, pp
import numpy as np

from bsp import app_registrar
from bsp.processes.simple_membrane_process import SimpleMembraneProcess


test_state_init_fp = '../test-membrane-initial.json'
oblate = "../fixtures/sample_meshes/oblate.ply"
prolate = "../fixtures/sample_meshes/prolate.ply"

membrane_config = {
    'characteristic_time_step': 2,
    'geometry': {
        'type': 'icosphere',
        'parameters': {
            'radius': 1.0,
            'subdivision': 2
        }
    },
    'tension_model': {
        'modulus': 0.1,
        'preferred_area': 12.4866
    },
    'osmotic_model': {
        'preferred_volume': 2.9321531433504737,
        'reservoir_volume': 0,
        'strength': 0.02,
        'volume': 2.9
    },
    'parameters': {
        'bending': {
            'Kbc': 8.22e-05
        }
    },
    'tolerance': 1e-11
}


membrane = SimpleMembraneProcess(config=membrane_config, core=app_registrar.core)

In [2]:
initial_state = membrane.initial_state()

pp(initial_state)

{ 'geometry': { 'faces': [ [0, 42, 44],
                           [12, 43, 42],
                           [14, 44, 43],
                           [42, 43, 44],
                           [11, 45, 47],
                           [13, 46, 45],
                           [12, 47, 46],
                           [45, 46, 47],
                           [5, 48, 50],
                           [14, 49, 48],
                           [13, 50, 49],
                           [48, 49, 50],
                           [12, 46, 43],
                           [13, 49, 46],
                           [14, 43, 49],
                           [46, 49, 43],
                           [0, 44, 52],
                           [14, 51, 44],
                           [16, 52, 51],
                           [44, 51, 52],
                           [5, 53, 48],
                           [15, 54, 53],
                           [14, 48, 54],
                           [53, 54, 48],
                    

In [3]:
membrane.initial_vertices.shape

(162, 3)

In [4]:
result = membrane.update(state=initial_state, interval=1)

Initialized NetCDF file at /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpc_bxb0df/traj.nc

t: 0, n: 0, isSmooth: 1
A, tension: 12.3298, 0.024456, V, pressure: 4.04704, -0.00259352, h: 1
nFaces: 320
minE: 0.275904; maxE: 0.32492
minA: 0.0360323; maxA: 0.0457144
E_total: 0.0331538
E_kin: 0
E_pot: 0.0331538
W_ext: 0
|e|Mech: 0.0499281
|e|Chem: 0
H: [1.00215,1.14736]
K: [1.00821,1.15383]
phi: [1,1]
sum_phi: 12.3298
H0: [0,0]

Reached time.

t: 2, n: 1, isSmooth: 1
A, tension: 12.1369, 0.0225084, V, pressure: 3.95233, -0.00237319, h: 0.992098
nFaces: 320
minE: 0.273743; maxE: 0.322461
minA: 0.0355158; maxA: 0.0450251
E_total: 0.0296345
E_kin: 0.00124641
E_pot: 0.0283881
W_ext: 0
|e|Mech: 0.00249281
|e|Chem: 0
H: [1.00609,1.16566]
K: [1.01155,1.19101]
phi: [1,1]
sum_phi: 12.1369
H0: [0,0]
Simulation finished, and data saved to /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpc_bxb0df
Closed NetCDF file


In [7]:
result.keys()

dict_keys(['geometry', 'protein_density', 'velocities', 'volume', 'preferred_volume', 'reservoir_volume', 'surface_area', 'net_forces'])

In [ ]:
input_composite_state = {
    'membrane': {
        '_type': 'process',
        'address': 'local:simple-membrane-process',
        'config': membrane_config,
        'inputs': {
            'geometry': ['geometry_store'],
            'velocities': ['velocities_store'],
            'protein_density': ['protein_density_store'],
            'preferred_volume': ['preferred_volume_store'],
            'reservoir_volume': ['reservoir_volume_store'],
            'surface_area': ['surface_area_store'],
            'osmotic_strength': ['osmotic_strength_store'],
        },
        'outputs': {
            'geometry': ['geometry_store'],
            'velocities': ['velocities_store'],
            'protein_density': ['protein_density_store'],
            'preferred_volume': ['preferred_volume_store'],
            'reservoir_volume': ['reservoir_volume_store'],
            'surface_area': ['surface_area_store'],
            'net_forces': ['net_forces_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'geometry': 'GeometryType',
                'velocities': 'VelocitiesType',
                'protein_density': 'ProteinDensityType',
                'preferred_volume': 'float',
                'reservoir_volume': 'float',
                'surface_area': 'float',
                'net_forces': 'MechanicalForcesType',
            }
        },
        'inputs': {
            'geometry': ['geometry_store'],
            'velocities': ['velocities_store'],
            'protein_density': ['protein_density_store'],
            'external_force': ['external_force_store'],
            'osmotic_parameters': ['osmotic_parameters_store'],
        }
    }
}